In [2]:
# --- Installer et importer les bibliothèques principales
!pip install pandas PyMuPDF langchain sentence-transformers faiss-cpu --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.0 MB/s eta 0:00:00


In [3]:
pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 910.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 72.2 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import os
import pandas as pd
import pdfplumber
import pickle

# --- Fonction de chunking (diviser le texte en morceaux)
def chunk_text(text, chunk_size=1000):
    """
    Divise le texte en morceaux de taille fixe (chunk_size).
    """
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
    return chunks

# --- Fonctions utiles pour charger les données

# Charger et structurer le CSV
def load_structured_csv(csv_path):
    df = pd.read_csv(csv_path)
    # Structurer chaque ligne comme une chaîne de texte "colonne: valeur"
    structured_rows = df.astype(str).apply(lambda row: " | ".join([f"{col}: {val}" for col, val in row.items()]), axis=1).tolist()
    return structured_rows

# Charger et extraire le texte d'un fichier PDF avec pdfplumber
def load_pdf_text_with_pdfplumber(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()  # Extrait le texte de chaque page
    return text

# Charger plusieurs fichiers PDF avec pdfplumber
def load_multiple_pdfs_with_pdfplumber(pdf_folder_path):
    texts = []
    for filename in os.listdir(pdf_folder_path):
        if filename.endswith(".pdf"):
            full_path = os.path.join(pdf_folder_path, filename)
            texts.append(load_pdf_text_with_pdfplumber(full_path))
    return texts

# --- Exemple d'utilisation

# Chemin vers les données du CSV (SDN List)
csv_path = "/content/drive/MyDrive/RAG_Project/data/csv/structured_sdn_list.csv"
structured_rows = load_structured_csv(csv_path)
print(f"{len(structured_rows)} entrées dans la SDN List.")

# Appliquer le chunking sur les documents CSV
chunked_structured_rows = []
for row in structured_rows:
    chunked_structured_rows.extend(chunk_text(row))  # Ajouter des chunks du CSV
print(f"{len(chunked_structured_rows)} chunks CSV générés.")

# Chemin vers les fichiers PDF
pdf_folder = "/content/drive/MyDrive/RAG_Project/data/pdfs/"  # Assure-toi que le chemin est correct
pdf_texts = load_multiple_pdfs_with_pdfplumber(pdf_folder)
print(f"{len(pdf_texts)} documents PDF chargés avec pdfplumber.")

# Appliquer le chunking sur les documents PDF
chunked_pdf_texts = []
for text in pdf_texts:
    chunked_pdf_texts.extend(chunk_text(text))  # Ajouter des chunks du PDF
print(f"{len(chunked_pdf_texts)} chunks PDF générés.")

# Optionnel : combiner les chunks du CSV et des PDFs dans une seule liste
all_documents = chunked_structured_rows + chunked_pdf_texts
print(f"Total documents (CSV + PDF) : {len(all_documents)}")

# Enregistrer les documents combinés dans un fichier pickle
with open('/content/drive/MyDrive/RAG_Project/models/all_documents.pkl', 'wb') as f:
    pickle.dump(all_documents, f)
print("✅ all_documents enregistré dans models/all_documents.pkl")


17635 entrées dans la SDN List.
17702 chunks CSV générés.


16 documents PDF chargés avec pdfplumber.
874 chunks PDF générés.
Total documents (CSV + PDF) : 18576
✅ all_documents enregistré dans models/all_documents.pkl


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# --- Exemple d'utilisation

# Charger les données du CSV (SDN List)
csv_path = "/content/drive/MyDrive/RAG_Project/data/csv/structured_sdn_list.csv"
structured_rows = load_structured_csv(csv_path)
print(f"{len(structured_rows)} entrées dans la SDN List.")

# Appliquer le chunking sur les documents CSV
chunked_structured_rows = []
for row in structured_rows:
    chunked_structured_rows.extend(chunk_text(row))  # Ajouter des chunks du CSV
print(f"{len(chunked_structured_rows)} chunks CSV générés.")

# Charger les textes des PDFs
pdf_folder = "/content/drive/MyDrive/RAG_Project/data/pdfs/"  # adapte le chemin
pdf_texts = load_multiple_pdfs_with_pdfplumber(pdf_folder)
print(f"{len(pdf_texts)} documents PDF chargés.")

# Appliquer le chunking sur les documents PDF
chunked_pdf_texts = []
for text in pdf_texts:
    chunked_pdf_texts.extend(chunk_text(text))  # Ajouter des chunks du PDF
print(f"{len(chunked_pdf_texts)} chunks PDF générés.")

# Optionnel : tout mettre ensemble
all_documents = chunked_structured_rows + chunked_pdf_texts
print(f"Total documents (CSV + PDF) : {len(all_documents)}")

# Enregistrer les documents combinés dans un fichier pickle
with open('/content/drive/MyDrive/RAG_Project/models/all_documents.pkl', 'wb') as f:
    pickle.dump(all_documents, f)
print("✅ all_documents enregistré dans models/all_documents.pkl")


17635 entrées dans la SDN List.
17702 chunks CSV générés.


16 documents PDF chargés.
874 chunks PDF générés.
Total documents (CSV + PDF) : 18576
✅ all_documents enregistré dans models/all_documents.pkl


In [10]:
import pickle

# Après avoir construit all_documents
with open('/content/drive/MyDrive/RAG_Project/models/all_documents.pkl', 'wb') as f:
    pickle.dump(all_documents, f)
print("✅ all_documents enregistré dans models/all_documents.pkl")


✅ all_documents enregistré dans models/all_documents.pkl
